In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
total_epoch = 30
batch_size = 100
learning_rate = 0.0002
n_hidden = 256
n_input = 28 * 28
n_noise = 128

In [3]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

In [4]:
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros([n_hidden]))
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [5]:
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.zeros([n_hidden]))
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.zeros([1]))

In [6]:
def generator(noise_z):
    hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
    return output

In [7]:
def discriminator(inputs):
    hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
    output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
    return output

In [8]:
def get_noise(batch_size, n_noise):
    return np.random.normal(size=(batch_size, n_noise))

In [9]:
G = generator(Z)
D_gene = discriminator(G)
D_real = discriminator(X)

In [10]:
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_gene))
loss_G = tf.reduce_mean(tf.log(D_gene))

In [11]:
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [12]:
train_D = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(
    -loss_G, var_list=G_var_list)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples / batch_size)
loss_val_D, loss_val_G = 0, 0

for epoch in range(total_epoch):
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        noise = get_noise(batch_size, n_noise)
        
        _, cost_val_D = sess.run([train_D, loss_D], feed_dict={X: batch_xs,
                                                               Z: noise})
        _, loss_val_G = sess.run([train_G, loss_G], feed_dict={Z: noise})
    
    print("Epoch: {:04d}".format(epoch), 
          "D loss: {:.4f}".format(loss_val_D), 
          "G loss: {:.4f}".format(loss_val_G))
    
    if epoch == 0 or (epoch + 1) % 10 == 0:
        sample_size = 10
        noise = get_noise(sample_size, n_noise)
        samples = sess.run(G, feed_dict={Z: noise})
        
        fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
        
        for i in range(sample_size):
            ax[i].set_axis_off()
            ax[i].imshow(np.reshape(samples[i], (28, 28)))
        
        plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), 
                    box_inches='tight')
        plt.close(fig)

print("Optimization completed!")

Epoch: 0000 D loss: 0.0000 G loss: -2.0373
Epoch: 0001 D loss: 0.0000 G loss: -1.9383
Epoch: 0002 D loss: 0.0000 G loss: -2.9324
Epoch: 0003 D loss: 0.0000 G loss: -1.7103
Epoch: 0004 D loss: 0.0000 G loss: -1.6131
Epoch: 0005 D loss: 0.0000 G loss: -2.0371
Epoch: 0006 D loss: 0.0000 G loss: -2.2540
Epoch: 0007 D loss: 0.0000 G loss: -2.9381
Epoch: 0008 D loss: 0.0000 G loss: -2.2841
Epoch: 0009 D loss: 0.0000 G loss: -2.2076
Epoch: 0010 D loss: 0.0000 G loss: -1.8699
Epoch: 0011 D loss: 0.0000 G loss: -2.0948
Epoch: 0012 D loss: 0.0000 G loss: -2.1473
Epoch: 0013 D loss: 0.0000 G loss: -2.0927
Epoch: 0014 D loss: 0.0000 G loss: -2.2307
Epoch: 0015 D loss: 0.0000 G loss: -2.1184
Epoch: 0016 D loss: 0.0000 G loss: -1.8685
Epoch: 0017 D loss: 0.0000 G loss: -2.1498
Epoch: 0018 D loss: 0.0000 G loss: -2.3776
Epoch: 0019 D loss: 0.0000 G loss: -2.1121
Epoch: 0020 D loss: 0.0000 G loss: -2.4402
Epoch: 0021 D loss: 0.0000 G loss: -2.5794
Epoch: 0022 D loss: 0.0000 G loss: -2.4957
Epoch: 0023

KeyboardInterrupt: 